In [5]:
import numpy as np
import tensorflow as tf
# import pyttsx3
# import matplotlib.pyplot as plt # Không dùng trong phiên bản này, có thể bỏ

In [6]:
# --- Dữ liệu huấn luyện (giữ nguyên) ---
X_train = np.array([
    [1, 1, 0, 1, 0, 0],  # Cúm
    [0, 1, 1, 0, 0, 0],  # Cảm lạnh
    [1, 1, 0, 0, 1, 0],  # COVID-19
    [0, 0, 1, 0, 0, 1]   # Dị ứng
], dtype=np.float32)
y_train = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)

# --- Các hằng số Tiếng Việt ---
TEN_BENH = ["Cúm", "Cảm lạnh", "COVID-19", "Dị ứng"]
TEN_TRIEU_CHUNG = ["Sốt", "Ho", "Hắt hơi", "Mệt mỏi", "Mất vị giác", "Ngứa mắt"]

XET_NGHIEM_KHUYEN_NGHI = {
    "Cúm": "Xét nghiệm Cúm A/B",
    "Cảm lạnh": "Lấy dịch mũi (thường không cần xét nghiệm chuyên biệt)",
    "COVID-19": "Xét nghiệm PCR hoặc test nhanh kháng nguyên",
    "Dị ứng": "Xét nghiệm dị ứng da hoặc xét nghiệm máu IgE"
}

THUOC_KHUYEN_NGHI = {
    "Cúm": "Oseltamivir (Tamiflu) nếu được chỉ định sớm, nghỉ ngơi, uống nhiều nước",
    "Cảm lạnh": "Nghỉ ngơi, uống nhiều nước, thuốc giảm triệu chứng (giảm đau, hạ sốt, thuốc ho)",
    "COVID-19": "Nghỉ ngơi, cách ly, Paracetamol nếu sốt/đau, theo dõi triệu chứng và tư vấn y tế",
    "Dị ứng": "Thuốc kháng histamine (ví dụ: Loratadine, Cetirizine), tránh xa tác nhân gây dị ứng"
}

In [ ]:
# --- Xây dựng và huấn luyện mô hình (giữ nguyên) ---
def build_model():
    inputs = tf.keras.Input(shape=(6,))
    x = tf.keras.layers.Dense(16, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.5)(x, training=True) # training=True quan trọng cho Monte Carlo Dropout
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x, training=True) # training=True quan trọng cho Monte Carlo Dropout
    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)

model = build_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=100, verbose=0)

# --- Chức năng Text-to-Speech (TTS) ---
# def speak_vietnamese(text):
#     engine = pyttsx3.init()
#     # Cố gắng tìm giọng nói tiếng Việt
#     voices = engine.getProperty('voices')
#     vn_voice_id = None
#     for voice in voices:
#         # Kiểm tra xem tên giọng nói có chứa 'vietnamese' hoặc 'An' (tên giọng Microsoft tiếng Việt)
#         if "vietnamese" in voice.name.lower() or "an" in voice.name.lower() or "vi-VN" in voice.id.lower():
#             vn_voice_id = voice.id
#             break
#         # Bạn có thể cần phải tìm ID cụ thể của giọng nói tiếng Việt trên máy của mình
#         # Ví dụ: for voice in voices: print(voice, voice.id) rồi chọn ID phù hợp.
    
#     if vn_voice_id:
#         engine.setProperty('voice', vn_voice_id)
#     else:
#         print("Cảnh báo: Không tìm thấy giọng nói tiếng Việt chuyên dụng. Sử dụng giọng mặc định.")
#         # Nếu không có giọng tiếng Việt, TTS có thể không hoạt động tốt.
#         # Cân nhắc sử dụng gTTS:
#         # from gtts import gTTS
#         # import os
#         # import playsound
#         # def speak_gtts(text_to_speak, lang='vi'):
#         #     try:
#         #         tts = gTTS(text=text_to_speak, lang=lang, slow=False)
#         #         filename = "temp_audio_vn.mp3"
#         #         tts.save(filename)
#         #         playsound.playsound(filename)
#         #         os.remove(filename)
#         #     except Exception as e:
#         #         print(f"Lỗi khi phát âm thanh bằng gTTS: {e}")
#         #         # Fallback nếu gTTS lỗi
#         #         engine.say(text_to_speak) # Thử lại với pyttsx3
#         #         engine.runAndWait()
#         # speak_gtts(text) # Thay thế dòng dưới nếu dùng gTTS
#         # return

#     engine.say(text)
#     engine.runAndWait()

# --- Dự đoán với độ không chắc chắn (Monte Carlo Dropout) (giữ nguyên) ---
def predict_with_uncertainty(model, x_input, n_iter=100):
    # x_input phải là một batch, ví dụ np.array([symptoms_array])
    # Keras model expects a batch of inputs.
    # If x_input is a single sample, it needs to be reshaped e.g., x_input.reshape(1, -1)
    # Hoặc đảm bảo x_input được truyền vào đã là dạng batch np.array([input_symptoms_list])
    
    # predictions = []
    # for _ in range(n_iter):
    #     predictions.append(model(x_input, training=True).numpy()) # training=True kích hoạt Dropout
    # predictions = np.array(predictions)
    
    # Cách viết gọn hơn và hiệu quả hơn một chút:
    preds_list = [model(x_input, training=True).numpy() for _ in range(n_iter)]
    preds = np.array(preds_list)


    mean_probs = preds.mean(axis=0)
    std_devs = preds.std(axis=0)
    return mean_probs, std_devs

# --- Xử lý đầu vào ngôn ngữ tự nhiên đơn giản ---
def interpret_yes_no_vietnamese(answer_text):
    answer = answer_text.strip().lower()
    positive_responses = ["có", "c", "yes", "y", "đúng", "phải", "roi", "co"] # "roi" cho "rồi", "co" cho "có" không dấu
    # negative_responses = ["không", "k", "no", "n", "sai", "chưa", "khong"] # "khong" cho "không" không dấu
    
    if answer in positive_responses:
        return 1
    # Mặc định là không nếu không phải là một trong các câu trả lời tích cực rõ ràng
    return 0

# --- Chạy Robot Trợ Lý Sức Khỏe Ảo ---
def run_virtual_robot_vn():
    intro_text = "Xin chào! Tôi là robot trợ lý sức khỏe ảo của bạn."
    print(intro_text)
    # speak_vietnamese(intro_text)

    prompt_instruction = "Vui lòng trả lời các câu hỏi sau với Có hoặc Không (hoặc các từ tương đương):"
    print(prompt_instruction)
    # speak_vietnamese(prompt_instruction) # Có thể bỏ qua nói câu này để nhanh hơn

    input_symptoms_values = []
    for trieu_chung in TEN_TRIEU_CHUNG:
        while True:
            prompt = f"Bạn có bị {trieu_chung.lower()} không? (Có/Không): "
            # speak_vietnamese(f"Bạn có bị {trieu_chung.lower()} không?") # Hỏi từng câu
            ans_text = input(prompt)
            if ans_text.strip(): # Đảm bảo người dùng nhập gì đó
                break
            print("Vui lòng nhập câu trả lời.")
        
        input_symptoms_values.append(interpret_yes_no_vietnamese(ans_text))

    input_array = np.array([input_symptoms_values], dtype=np.float32)

    # Dự đoán với độ không chắc chắn
    mean_probabilities, std_dev_probabilities = predict_with_uncertainty(model, input_array, n_iter=100)

    # Lấy chỉ số của bệnh có xác suất cao nhất
    most_likely_index = np.argmax(mean_probabilities[0]) # mean_probabilities là [[p1, p2, p3, p4]]
    diagnosis = TEN_BENH[most_likely_index]
    
    print("\n--- Chẩn đoán với Xác suất và Độ không chắc chắn ---")
    for i, benh in enumerate(TEN_BENH):
        print(f"{benh}: Xác suất = {mean_probabilities[0][i]:.3f}, Độ không chắc chắn (StdDev) = {std_dev_probabilities[0][i]:.3f}")

    diagnosis_text = f"Dựa trên các triệu chứng bạn cung cấp, có khả năng bạn bị {diagnosis}."
    print(f"\nChẩn đoán sơ bộ: {diagnosis} (Độ không chắc chắn cho chẩn đoán này: ±{std_dev_probabilities[0][most_likely_index]:.3f})")
    # speak_vietnamese(diagnosis_text)

    if diagnosis in XET_NGHIEM_KHUYEN_NGHI:
        recommendation_text = f"Để xác định chính xác hơn, bạn nên cân nhắc thực hiện: {XET_NGHIEM_KHUYEN_NGHI[diagnosis]}."
        print(recommendation_text)
        # speak_vietnamese(recommendation_text)
    
    if diagnosis in THUOC_KHUYEN_NGHI:
        medication_text = f"Một số khuyến nghị về điều trị/thuốc cho {diagnosis.lower()} là: {THUOC_KHUYEN_NGHI[diagnosis]}."
        print(medication_text)
        # speak_vietnamese(medication_text)
        
    disclaimer_text = "Lưu ý: Đây chỉ là chẩn đoán sơ bộ dựa trên mô hình AI. Bạn nên tham khảo ý kiến của bác sĩ để có chẩn đoán chính xác và kế hoạch điều trị phù hợp."
    print(f"\n{disclaimer_text}")
    # speak_vietnamese(disclaimer_text)

if __name__ == "__main__":
    run_virtual_robot_vn()

Xin chào! Tôi là robot trợ lý sức khỏe ảo của bạn.
Vui lòng trả lời các câu hỏi sau với Có hoặc Không (hoặc các từ tương đương):

--- Chẩn đoán với Xác suất và Độ không chắc chắn ---
Cúm: Xác suất = 0.258, Độ không chắc chắn (StdDev) = 0.093
Cảm lạnh: Xác suất = 0.253, Độ không chắc chắn (StdDev) = 0.063
COVID-19: Xác suất = 0.223, Độ không chắc chắn (StdDev) = 0.069
Dị ứng: Xác suất = 0.266, Độ không chắc chắn (StdDev) = 0.056

Chẩn đoán sơ bộ: Dị ứng (Độ không chắc chắn cho chẩn đoán này: ±0.056)
Để xác định chính xác hơn, bạn nên cân nhắc thực hiện: Xét nghiệm dị ứng da hoặc xét nghiệm máu IgE.
Một số khuyến nghị về điều trị/thuốc cho dị ứng là: Thuốc kháng histamine (ví dụ: Loratadine, Cetirizine), tránh xa tác nhân gây dị ứng.

Lưu ý: Đây chỉ là chẩn đoán sơ bộ dựa trên mô hình AI. Bạn nên tham khảo ý kiến của bác sĩ để có chẩn đoán chính xác và kế hoạch điều trị phù hợp.


: 